In [1]:
from dataset import CoverDataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from models import get_models
import albumentations as A
import torch
from defs import *

/root/anaconda3/envs/album/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
dataset = CoverDataset(DATA_PATH, IMSIZE, None, True, True)
train_dataloader = DataLoader(
    dataset, batch_size=TRAIN_BATCH_SIZE, shuffle=True, drop_last=True
)

Loading dataset: 79734it [05:09, 257.53it/s]


In [11]:
a = next(iter(train_dataloader))

In [34]:
for idx, i in enumerate(range(2)):
    print(idx)

0
1


In [17]:
model = get_models(IMSIZE)

In [21]:
output = model(a.float(), 0)

torch.Size([4, 3, 64, 64])